In [1]:
# from openai_sdk_resume_assistant.tool_agent import tool_agent, connect_to_servers, tool_mcp_servers, params_list
from openai_sdk_resume_assistant.tool_agent import get_tool_agent, params_list, get_mcp_servers
from dotenv import load_dotenv
from agents import Runner, set_default_openai_api, set_default_openai_client, set_tracing_disabled
from openai_sdk_resume_assistant.client import AzureAIClient
from openai import AsyncAzureOpenAI
from agents.mcp import MCPServerStdio

In [2]:
load_dotenv(override=True)

client = AzureAIClient()
openai_client = AsyncAzureOpenAI(
    azure_endpoint = client.azure_endpoint,
    api_version = client.api_version,
    azure_ad_token_provider = client.token_provider
)

/usr/bin/xdg-open: 882: x-www-browser: not found
/usr/bin/xdg-open: 882: firefox: not found
/usr/bin/xdg-open: 882: iceweasel: not found
/usr/bin/xdg-open: 882: seamonkey: not found
/usr/bin/xdg-open: 882: mozilla: not found
/usr/bin/xdg-open: 882: epiphany: not found
/usr/bin/xdg-open: 882: konqueror: not found
/usr/bin/xdg-open: 882: chromium: not found
/usr/bin/xdg-open: 882: chromium-browser: not found
/usr/bin/xdg-open: 882: google-chrome: not found
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize?client_id=04b07795-8ddb-461a-bbee-02f9e1bf7b46&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8400&scope=https%3A%2F%2Fcognitiveservices.azure.com%2F.default+offline_a

In [3]:
# Set default client and configs
set_default_openai_client(openai_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

In [ ]:
send_email_params = {"command": "uv", "args" : ["run", "agent_tools.py"]}

In [ ]:
async with MCPServerStdio(params=send_email_params, client_session_timeout_seconds=60) as server:
    own_tools = await server.list_tools()

own_tools

In [ ]:
## Using AsyncExitStack() as stack:
from contextlib import AsyncExitStack

# mcp_servers  = []
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params=params, client_session_timeout_seconds=60)) for params in params_list]


In [ ]:
mcp_servers

### Testing from modules on creating an agent with MCP servers

In [4]:
user_input = "Hey can you search the latest news about Anthropic the company and summarize it for me to a file called Anthropic_update.md?"

In [ ]:
from agents import Agent
from openai_sdk_resume_assistant.tool_agent import TOOL_AGENT_INSTRUCTIONS
from contextlib import AsyncExitStack

In [ ]:
async with AsyncExitStack() as stack:
        tool_mcp_servers = [
            await stack.enter_async_context(MCPServerStdio(params=params, client_session_timeout_seconds=60)) for params in params_list
            ]
        agent = Agent(
                name = "TestAgent",
                model="gpt-4o",
                instructions=TOOL_AGENT_INSTRUCTIONS,
                mcp_servers=tool_mcp_servers,
        )
        reponse = await Runner.run(agent, input=user_input)

In [ ]:
print(f"Response: {reponse.final_output}")

### Attempting directly from tool_agent.py

In [5]:

user_input3 = "Find some relevant information about Anthropic the company and summarize it to the file called Anthropic_update.md"

In [ ]:
async with get_mcp_servers() as servers:
    agent = await get_tool_agent(servers)

    response = await Runner.run(agent, input=user_input3)

print("Response:", response.final_output)

In [7]:
params_list

[{'command': 'npx', 'args': ['@playwright/mcp@latest']},
 {'command': 'npx',
  'args': ['-y',
   '@modelcontextprotocol/server-filesystem',
   '/mnt/c/Users/edan/repos/personal_projects/AI/Agentic_AI_Projects/openai_sdk_resume_assistant/src/openai_sdk_resume_assistant/file_storage']}]

In [8]:
mcp_servers = await get_mcp_servers()
agent = await get_tool_agent(mcp_servers)

response = await Runner.run(agent, input=user_input3)

print("Response:", response.final_output)

TypeError: object _AsyncGeneratorContextManager can't be used in 'await' expression

In [12]:
from contextlib import AsyncExitStack
async with AsyncExitStack() as stack:
        tool_mcp_servers = [
            await stack.enter_async_context(MCPServerStdio(params=params, client_session_timeout_seconds=60)) for params in params_list
            ]
        tool_agent = await get_tool_agent(tool_mcp_servers)
        response = await Runner.run(tool_agent, input=user_input3)

In [13]:
print("Response:", response.final_output)

Response: The file "Anthropic_update.md" already contains a comprehensive summary of the Anthropic company. Here is the information captured in the file:

---

# Anthropic Company Overview

## Brief Summary

Anthropic is an AI safety and research company focused on making large-scale AI systems more interpretable, steerable, and robust. Founded by former members of OpenAI, Anthropic aims to address the challenges and ensure safe deployment of AI.

## History and Background

Anthropic was founded in 2021 by Daniela and Dario Amodei along with several other members who previously worked at OpenAI. The company emphasizes research in AI safety and the development of safety standards.

## Key Focus Areas

1. **AI Alignment**: Developing AI systems that align with human intentions and safety protocols.
2. **Interpretability**: Creating models that can explain their processes and decisions.
3. **Robustness**: Ensuring AI systems can perform reliably under various circumstances.

## Partnershi

In [14]:
from contextlib import asynccontextmanager

@asynccontextmanager
async def mcp_servers_context(params_list):
    async with AsyncExitStack() as stack:
        servers = [
            await stack.enter_async_context(MCPServerStdio(params=params, client_session_timeout_seconds=60))
            for params in params_list
        ]
        yield servers

In [15]:
async with mcp_servers_context(params_list) as servers:
    agent = await get_tool_agent(servers)
    response = await Runner.run(agent, input=user_input3)

In [17]:
print("Response: ", response.final_output)

Response:  I have created a file named `Anthropic_update.md` with a note about the current inability to retrieve updated information about Anthropic due to browser installation issues. You can find this file in the directory `/mnt/c/Users/edan/repos/personal_projects/AI/Agentic_AI_Projects/openai_sdk_resume_assistant/src/openai_sdk_resume_assistant/file_storage`. Please let me know if there is anything else I can assist you with!
